# Working with Langchain

In [ ]:
# pip install langchain langchain-experimental langchain-openai

### Chatmodels, Pormpt Templates and Parsers

In [3]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(api_version="2024-12-01-preview",model='telcogpt')

prompt = "Tell me a joke about mouse"
model.invoke(prompt)

AIMessage(content='What do you call a mouse that can sing?\n\nA little mouse-ician! 🐭🎶', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 13, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-BWxY4BWVOPmNdGjw77QdUwietj0Vv', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'low'}, 'self_harm': {'filtered': False, 'severity': 'safe

In [4]:
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}"

prompt = ChatPromptTemplate([("system",system_template),("user","{text}")])

prompt

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following into {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [7]:
prompt.input_variables

['language', 'text']

In [9]:
prompt.invoke({"language":"hindi","text":"HOW ARE YOU?"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into hindi', additional_kwargs={}, response_metadata={}), HumanMessage(content='HOW ARE YOU?', additional_kwargs={}, response_metadata={})])

In [14]:
# PARSERS
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
final_prompt = prompt.invoke({"language":"hindi","text":"HOW ARE YOU?"})
op = model.invoke(final_prompt)
op

AIMessage(content='आप कैसे हैं?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 20, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-BWxeZnIeAI1qdNPUIU6rtYZfQ6jnt', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 

In [15]:
parser.invoke(op)

'आप कैसे हैं?'